# Gerando a estrutura organizacional em rede

In [1]:
!pip install igraph
!pip install pycairo
!pip install cairocffi
!pip install python-igraph

In [21]:
import time
import datetime as dt
import pickle
import igraph as ig
print("igraph-version : ",ig.__version__)
import cairo
import pandas as pd

base_path = "C:/projetos/prova-de-conceito/"
df = pd.read_csv(base_path + "dados/dou/2022/2022.csv")
df["entidades"] = df["emissor"].str.split("/")
print(len(df))
df.head(10)

    


igraph-version :  0.10.1
580407


,id,tipo,emissor,data,responsavel,cargo,entidades
0,S01012022/1020_20220124_14186551,Lei,Atos do Poder Legislativo,24/01/2022,NaN,NaN,[Atos do Poder Legislativo]
1,S01012022/1020_20220124_14186554,Lei,Atos do Poder Legislativo,24/01/2022,NaN,NaN,[Atos do Poder Legislativo]
2,S01012022/1020_20220124_14186574,Lei,Atos do Poder Legislativo,24/01/2022,NaN,NaN,[Atos do Poder Legislativo]
3,S01012022/1020_20220124_14186593,Lei,Atos do Poder Legislativo,24/01/2022,NaN,NaN,[Atos do Poder Legislativo]
4,S01012022/1020_20220124_14186598,Lei,Atos do Poder Legislativo,24/01/2022,NaN,NaN,[Atos do Poder Legislativo]
5,S01012022/1020_20220124_14186613,Lei,Atos do Poder Legislativo,24/01/2022,NaN,NaN,[Atos do Poder Legislativo]
6,S01012022/1020_20220124_14186616,Lei,Atos do Poder Legislativo,24/01/2022,NaN,NaN,[Atos do Poder Legislativo]
7,S01012022/1020_20220124_14186619,Lei,Atos do Poder Legislativo,24/01/2022,NaN,NaN,[Atos do Poder Legislativo]
8,S01012022/1020_20220124_14186658,Lei,Atos do Poder Legislativo,24/01/2022,NaN,NaN,[Atos do Poder Legislativo]
9,S01012022/1020_20220124_14206301,Lei,Atos do Poder Legislativo,24/01/2022,Paulo Guedes,NaN,[Atos do Poder Legislativo]


In [41]:
g = ig.Graph(directed=True)
g["name"]="Unidades Organizacionais da Administração Pública"
g.vs["min_date"]=[]
g.vs["max_date"]=[]
g.vs["weight"]=[]
g.vs["avg_time"]=[]
g.vs["responsible"]=[]
g.es["weight"] = []


start_time = time.time()


for index, row in df.iterrows(): 
    split_data = row["data"].split("/")
    dia = split_data[0]
    mes = split_data[1]
    ano = split_data[2]
    d = dt.date(int(ano),int(mes),int(dia))
    for index,ent in enumerate(row["entidades"]):
        
        try:
            vert = g.vs.find(ent) 
            vert["weight"] = vert["weight"] + 1
            if ( d < vert["min_date"] ):
                vert["min_date"] = d
            if ( d > vert["max_date"] ):
                vert["max_date"] = d
            vert["avg_time"] = (vert["max_date"] - vert["min_date"]) \
            / vert["weight"]
            if(index == len(row["entidades"])-1) and \
            type(vert["responsible"]) == 'list':
                if not (pd.isnull(row["responsavel"] )):
                    if not (pd.isnull(row["cargo"] )):
                        vert["responsible"] = vert["responsible"].\
                        append(row["responsavel"]+"|" +row["cargo"])
                    else:
                        vert["responsible"] = vert["responsible"].\
                        append(row["responsavel"])
            
        except:
            g.add_vertices([ent])
            vert = g.vs.find(ent)            
            vert["min_date"] =  d
            vert["max_date"] =  d
            vert["weight"] =  1
            vert["avg_time"]=0.0
            if(index == len(row["entidades"])-1):
                if not (pd.isnull(row["responsavel"] )):
                    vert["responsible"]=[row["responsavel"]]
                    if not (pd.isnull(row["cargo"] )):
                        vert["responsible"]=[row["responsavel"]+"|"\
                                                +row["cargo"]]

    for (index,ent) in enumerate(row['entidades']):
        if len(row['entidades']) > index+1:
            eg = g.es.select(lambda \
                            edge: edge.source_vertex["name"]  == ent and \
                            edge.target_vertex["name"] == row['entidades'][index+1] ) 
            if len(eg) > 0:
                eg[0]["weight"] = eg[0]["weight"] +  1
                if ( d < eg[0]["min_date"] ):
                    eg[0]["min_date"] = d                    
                if ( d > eg[0]["max_date"] ):
                    eg[0]["max_date"] = d
                eg[0]["avg_time"] = (eg[0]["max_date"] - eg[0]["min_date"])\
                / eg[0]["weight"]
                
            if not (len(eg) > 0):
                g.add_edges([(ent , row['entidades'][index+1])])
                eg = g.es.select(lambda \
                            edge: edge.source_vertex["name"]  == ent and \
                            edge.target_vertex["name"] == row['entidades'][index+1] ) 
                eg[0]["min_date"] =  d
                eg[0]["max_date"] =  d
                eg[0]["weight"] =  1
                eg[0]["avg_time"]=0.0
                
print(g.summary())

for vx in g.vs[:10]:
    print((vx["responsible"]))
    print(type(vx["responsible"]) == 'list')
    print(vx.attributes()  )
#for es in g.es[:10]:
#    print(es.attributes()) 
#    print(es.source_vertex["name"] + "-->" + es.target_vertex["name"])

with open("grafo.pkl", 'wb') as pickle_file:
    pickle.dump(g, pickle_file)
    
print("--- %s seconds ---" % (time.time() - start_time))
print("---",(time.time() - start_time)/60, "minutes ---")



IGRAPH DNW- 14330 16191 -- Unidades Organizacionais da Administração Pública
+ attr: name (g), avg_time (v), max_date (v), min_date (v), name (v), responsible (v), weight (v), avg_time (e), max_date (e), min_date (e), weight (e)
None
False
{'min_date': datetime.date(2022, 1, 4), 'max_date': datetime.date(2022, 7, 29), 'weight': 182, 'avg_time': datetime.timedelta(days=1, seconds=11393, microseconds=406593), 'responsible': None, 'name': 'Atos do Poder Legislativo'}
None
False
{'min_date': datetime.date(2022, 1, 3), 'max_date': datetime.date(2022, 7, 29), 'weight': 41567, 'avg_time': datetime.timedelta(seconds=430, microseconds=264392), 'responsible': None, 'name': 'Ministério da Defesa'}
None
False
{'min_date': datetime.date(2022, 1, 3), 'max_date': datetime.date(2022, 7, 29), 'weight': 21818, 'avg_time': datetime.timedelta(seconds=819, microseconds=726831), 'responsible': None, 'name': 'Comando do Exército'}
None
False
{'min_date': datetime.date(2022, 1, 3), 'max_date': datetime.date(2

In [78]:
with cairo.SVGSurface("example.svg", 200, 200) as surface:
    context = cairo.Context(surface)
    x, y, x1, y1 = 0.1, 0.5, 0.4, 0.9
    x2, y2, x3, y3 = 0.6, 0.1, 0.9, 0.5
    context.scale(200, 200)
    context.set_line_width(0.04)
    context.move_to(x, y)
    context.curve_to(x1, y1, x2, y2, x3, y3)
    context.stroke()
    context.set_source_rgba(1, 0.2, 0.2, 0.6)
    context.set_line_width(0.02)
    context.move_to(x, y)
    context.line_to(x1, y1)
    context.move_to(x2, y2)
    context.line_to(x3, y3)
    context.stroke()